<a href="https://colab.research.google.com/github/pratikkotian04/NLP/blob/master/OpenGPT2_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/openai/gpt-2.git

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [2]:
import os
os.chdir("gpt-2")

In [3]:
!pip3 install tensorflow==1.13.2

In [4]:
!pip3 install -r requirements.txt

In [5]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 1.31Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 70.5Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.40Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:16, 87.4Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 10.4Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 60.3Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 63.9Mit/s]                                                       


In [6]:

!export PYTHONIOENCODING=UTF-8

In [7]:
os.chdir('src')

In [8]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [10]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [ ]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.random.categorical instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> I went to the hospital and saw
======================================== SAMPLE 1 ========================================
 Cedric McLendon. I think it was right around July; there was no one home for days. During the bus ride to the hospital, all this woke me up! You see, switching that over caused me to continuously forget until I reunited all our memories. Now I just want to be home right now.

What truly gave me the strength and resolve to transform was the directive of Fred Kempen. Yes, I wanted to be back out there, and it was possible now that maybe something good could come out of found I.

Photographs courtesy Nicolas 